# Basics

In [1]:
import pickle
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import joblib
import math
import numpy as np
from keras.models import load_model
import matplotlib.colors
from matplotlib.colors import ListedColormap
%matplotlib inline

In [2]:
my_list=pickle.load(open('./data/real_time_data3.pkl','rb'))

In [3]:
final_df = pd.DataFrame(columns=["symbol", "day", "HrMin", "Second", "timestamp", "price"])
for i in range(len(my_list)):
    temp_df = my_list[i]
    final_df = pd.concat([temp_df, final_df])

final_df.drop_duplicates()
final_df['price'] = final_df['price'].astype(float)

# NVDA

In [4]:
NVDA = final_df[final_df['symbol']=='NVDA']
NVDA = NVDA.drop_duplicates(subset=['day', 'HrMin'] , keep='first')
NVDA= NVDA.reset_index(drop = True)
NVDA = NVDA.set_index('DateTime')

In [5]:
NVDA = NVDA[NVDA.index<='2023-10-17 06:59:10']

In [6]:
NVDA

,symbol,day,HrMin,Second,timestamp,price
DateTime,,,,,,
2023-10-17 06:34:55,NVDA,10/17/23,06:34,55,063000,456.00
2023-10-17 06:35:09,NVDA,10/17/23,06:35,09,776000,455.97
2023-10-17 06:36:00,NVDA,10/17/23,06:36,00,043000,456.12
2023-10-17 06:37:01,NVDA,10/17/23,06:37,01,318000,456.23
2023-10-17 06:38:00,NVDA,10/17/23,06:38,00,934000,456.16
2023-10-17 06:39:00,NVDA,10/17/23,06:39,00,383000,456.40
2023-10-17 06:40:07,NVDA,10/17/23,06:40,07,214000,456.52
2023-10-17 06:41:01,NVDA,10/17/23,06:41,01,358000,456.50
2023-10-17 06:42:05,NVDA,10/17/23,06:42,05,216000,456.19


In [7]:
file_path = r'.\models\nvda_lstm_model.h5'

# Load the MinMaxScaler from the file
scaler = joblib.load(r'.\models\nvda_minmax_scaler.pkl')

# Load the pre-trained model
model = load_model(r'.\models\nvda_lstm_model.h5')

model.compile(optimizer='adam', loss='mean_squared_error')

In [8]:
open_prices = NVDA['price']
values = open_prices.values
training_data_len = math.ceil(len(values)* 0.8)

scaled_data = scaler.fit_transform(values.reshape(-1,1))
train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(6, len(train_data)):
    x_train.append(train_data[i-6:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [9]:
test_data = scaled_data[training_data_len-6: , : ]
x_test = []
y_test = values[training_data_len:]

for i in range(6, len(test_data)):
    x_test.append(test_data[i-6:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [10]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint(file_path, monitor='loss', mode='min', verbose=1, save_best_only=True)
model.fit(x_train, y_train,  epochs=10, batch_size=64, callbacks=[es, mc])

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.0477
Epoch 1: loss improved from inf to 0.04765, saving model to .\models\nvda_lstm_model.h5
1/1 [==============================] - 3s 3s/step - loss: 0.0477
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.0425
Epoch 2: loss improved from 0.04765 to 0.04248, saving model to .\models\nvda_lstm_model.h5
1/1 [==============================] - 0s 29ms/step - loss: 0.0425
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.0386
Epoch 3: loss improved from 0.04248 to 0.03863, saving model to .\models\nvda_lstm_model.h5
1/1 [==============================] - 0s 16ms/step - loss: 0.0386
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.0355
Epoch 4: loss improved from 0.03863 to 0.03552, saving model to .\models\nvda_lstm_model.h5
1/1 [==============================] - 0s 31ms/step - loss: 0.0355
Epoch 5/10
1/1 [==============================] - ETA: 0s - loss: 0.0330
Epoch

C:\Users\nisha\anaconda3\envs\test\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - ETA: 0s - loss: 0.0277
Epoch 8: loss improved from 0.02910 to 0.02767, saving model to .\models\nvda_lstm_model.h5
1/1 [==============================] - 0s 22ms/step - loss: 0.0277
Epoch 9/10
1/1 [==============================] - ETA: 0s - loss: 0.0264
Epoch 9: loss improved from 0.02767 to 0.02644, saving model to .\models\nvda_lstm_model.h5
1/1 [==============================] - 0s 16ms/step - loss: 0.0264
Epoch 10/10
1/1 [==============================] - ETA: 0s - loss: 0.0254
Epoch 10: loss improved from 0.02644 to 0.02538, saving model to .\models\nvda_lstm_model.h5
1/1 [==============================] - 0s 32ms/step - loss: 0.0254


In [11]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

1/1 [==============================] - 1s 546ms/step


0.25364501953125684

## Future Predictions

In [12]:
# Create a new array to store your predictions.
future_predictions = []

# The number of future time steps you want to predict (in this case, 5 steps).
num_steps_to_predict = 5

# Start with the last available data point.
last_data_point = x_train[-1]

In [13]:
for _ in range(num_steps_to_predict):
    # Use the model to predict the next time step.
    next_prediction = model.predict(np.array([last_data_point]))
    
    # Store the prediction in your results array.
    future_predictions = np.concatenate((future_predictions, next_prediction[0]))
    
    # Update the last_data_point for the next iteration.
    last_data_point = np.concatenate((last_data_point, next_prediction))
    np.delete(last_data_point, 0)

print(future_predictions)

1/1 [==============================] - 0s 16ms/step
[0.46387893 0.47154993 0.4773674  0.48139209 0.48382381]


In [14]:
nvda_predicted = scaler.inverse_transform(future_predictions.reshape(-1,1))

In [15]:
nvda_predictions = pd.DataFrame(nvda_predicted, index= 
                                        [
                                            '2023-10-17 07:00:00',
                                        '2023-10-17 07:01:00',
                                        '2023-10-17 07:02:00', 
                                        '2023-10-17 07:03:00', 
                                        '2023-10-17 07:04:00' 
                                         ], columns=["price"])

# META

In [16]:
META = final_df[final_df['symbol']=='META']
META =META.drop_duplicates(subset=['day', 'HrMin'] , keep='first')
META=META.reset_index(drop = True)
META= META.set_index('DateTime')

In [17]:
META = META[META.index<='2023-10-17 06:59:10']

In [18]:
file_path = '.\models\meta_lstm_model.h5'

# Load the MinMaxScaler from the file
scaler = joblib.load('.\models\meta_minmax_scaler.pkl')

# Load the pre-trained model
model = load_model('.\models\meta_lstm_model.h5')

model.compile(optimizer='adam', loss='mean_squared_error')

In [19]:
open_prices = META['price']
values = open_prices.values
training_data_len = math.ceil(len(values)* 0.8)

scaled_data = scaler.fit_transform(values.reshape(-1,1))
train_data = scaled_data[0: training_data_len, :]

x_train = []
y_train = []

for i in range(6, len(train_data)):
    x_train.append(train_data[i-6:i, 0])
    y_train.append(train_data[i, 0])
    
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

In [20]:
test_data = scaled_data[training_data_len-6: , : ]
x_test = []
y_test = values[training_data_len:]

for i in range(6, len(test_data)):
    x_test.append(test_data[i-6:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [21]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint(file_path, monitor='loss', mode='min', verbose=1, save_best_only=True)
model.fit(x_train, y_train,  epochs=10, batch_size=64, callbacks=[es, mc])

Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 0.0255
Epoch 1: loss improved from inf to 0.02551, saving model to .\models\meta_lstm_model.h5
1/1 [==============================] - 3s 3s/step - loss: 0.0255
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 0.0195
Epoch 2: loss improved from 0.02551 to 0.01954, saving model to .\models\meta_lstm_model.h5
1/1 [==============================] - 0s 31ms/step - loss: 0.0195
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 0.0182
Epoch 3: loss improved from 0.01954 to 0.01823, saving model to .\models\meta_lstm_model.h5
1/1 [==============================] - 0s 33ms/step - loss: 0.0182
Epoch 4/10
1/1 [==============================] - ETA: 0s - loss: 0.0178
Epoch 4: loss improved from 0.01823 to 0.01784, saving model to .\models\meta_lstm_model.h5
1/1 [==============================] - 0s 15ms/step - loss: 0.0178
Epoch 5/10
1/1 [==============================] - ETA: 0s - loss: 0.0169
Epoch

C:\Users\nisha\anaconda3\envs\test\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 8/10
1/1 [==============================] - ETA: 0s - loss: 0.0133
Epoch 8: loss improved from 0.01424 to 0.01334, saving model to .\models\meta_lstm_model.h5
1/1 [==============================] - 0s 25ms/step - loss: 0.0133
Epoch 9/10
1/1 [==============================] - ETA: 0s - loss: 0.0129
Epoch 9: loss improved from 0.01334 to 0.01288, saving model to .\models\meta_lstm_model.h5
1/1 [==============================] - 0s 15ms/step - loss: 0.0129
Epoch 10/10
1/1 [==============================] - ETA: 0s - loss: 0.0126
Epoch 10: loss improved from 0.01288 to 0.01263, saving model to .\models\meta_lstm_model.h5
1/1 [==============================] - 0s 16ms/step - loss: 0.0126


In [22]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

1/1 [==============================] - 1s 512ms/step


0.027257080078129547

## Future Predictions

In [23]:
# Create a new array to store your predictions.
future_predictions = []

# The number of future time steps you want to predict (in this case, 5 steps).
num_steps_to_predict = 5

# Start with the last available data point.
last_data_point = x_train[-1]

In [24]:
for _ in range(num_steps_to_predict):
    # Use the model to predict the next time step.
    next_prediction = model.predict(np.array([last_data_point]))
    
    # Store the prediction in your results array.
    future_predictions = np.concatenate((future_predictions, next_prediction[0]))
    
    # Update the last_data_point for the next iteration.
    last_data_point = np.concatenate((last_data_point, next_prediction))
    np.delete(last_data_point, 0)

print(future_predictions)

1/1 [==============================] - 0s 16ms/step
[0.33636588 0.32357308 0.31668985 0.31493947 0.31637502]


In [25]:
meta_predicted = scaler.inverse_transform(future_predictions.reshape(-1,1))

In [26]:
meta_predictions = pd.DataFrame(meta_predicted, index= 
                                        ['2023-10-17 07:00:00',
                                        '2023-10-17 07:01:00',
                                        '2023-10-17 07:02:00', 
                                        '2023-10-17 07:03:00', 
                                        '2023-10-17 07:04:00' 
                                         ], columns=["price"])

# Visual

In [27]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [30]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=META.index,
                         y=META['price'],
                         opacity=0.7,
                         line=dict(color='blue', width=2),
                         name='Meta - Actual'))
fig.add_trace(go.Scatter(x=meta_predictions.index,
                         y=meta_predictions['price'],
                         opacity=0.7,
                         line=dict(color='red', width=2),
                         name='Meta - prediction'))
# Show the figure
fig.show()

In [31]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=NVDA.index,
                         y=NVDA['price'],
                         opacity=0.7,
                         line=dict(color='blue', width=2),
                         name='NVDA - Actual'))
fig.add_trace(go.Scatter(x=nvda_predictions.index,
                         y=nvda_predictions['price'],
                         opacity=0.7,
                         line=dict(color='red', width=2),
                         name='NVDA - prediction'))
# Show the figure
fig.show()

In [63]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces

fig.add_trace(go.Scatter(x=META.index,
                         y=META['price'],
                         opacity=0.7,
                         line=dict(color='green', width=2),
                         name='Meta'), secondary_y=False,)

fig.add_trace(go.Scatter(x=meta_predictions.index,
                         y=meta_predictions['price'],
                         opacity=0.7,
                         line=dict(color='green', width=2, dash='dot'),
                         name='Meta - prediction'), secondary_y=False,)

fig.add_trace(go.Scatter(x=NVDA.index,
                         y=NVDA['price'],
                         opacity=0.7,
                         line=dict(color='blue', width=2),
                         name='NVDA'), secondary_y=True,)

fig.add_trace(go.Scatter(x=nvda_predictions.index,
                         y=nvda_predictions['price'],
                         opacity=0.7,
                         line=dict(color='blue', width=2,  dash='dot'),
                         name='NVDA - prediction') , secondary_y=True,)


# Add figure title
fig.update_layout(height=500, 
                  width=1000,
                  xaxis_rangeslider_visible=False,
                title="Price<br><sup>Predictions</sup>",
                 title_font_color="#000", title_font_size = 24)



# Set y-axes titles
fig.update_yaxes(title_text="<b>META</b> prices", secondary_y=False, title_font_color="green")
fig.update_yaxes(title_text="<b>NVDA</b> prices", secondary_y=True, title_font_color="blue")
fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])], showgrid=False)
fig.update_xaxes(visible=True)
fig.update_layout({
    'plot_bgcolor': 'rgb(247,247,247)',
    'paper_bgcolor': 'rgb(247,247,247)'
})

fig.show()

In [66]:
# save data for streamlit

path_file = './data'
ticker_file = 'meta_data.pkl'
pickle_out = open(path_file + '/' + ticker_file, "wb")
pickle.dump(META, pickle_out)
pickle_out.close()

ticker_file = 'nvda_data.pkl'
pickle_out = open(path_file + '/' + ticker_file, "wb")
pickle.dump(NVDA, pickle_out)
pickle_out.close()

ticker_file = 'meta_pred.pkl'
pickle_out = open(path_file + '/' + ticker_file, "wb")
pickle.dump(meta_predictions, pickle_out)
pickle_out.close()

ticker_file = 'nvda_pred.pkl'
pickle_out = open(path_file + '/' + ticker_file, "wb")
pickle.dump(nvda_predictions, pickle_out)
pickle_out.close()